In [2]:
using Distributions, LightGraphs, DataFrames, RCall, ProgressMeter

In [3]:
srand(20130810)

MersenneTwister(UInt32[0x01332bfa], Base.dSFMT.DSFMT_state(Int32[-1772545288, 1073534108, 1077066014, 1072915095, -2146195133, 1072843413, 301764553, 1073404181, 750472136, 1073628106  …  -1491411563, 1073194977, 716119449, 1072893711, 1632331784, 758890923, 1433693833, -13012230, 382, 0]), [1.26294, 1.899, 1.73924, 1.65031, 1.16468, 1.08045, 1.90283, 1.04602, 1.5813, 1.89209  …  1.43725, 1.65819, 1.31951, 1.37285, 1.21427, 1.81925, 1.90415, 1.70235, 1.31837, 1.08371], 382)

In [4]:
mutable struct Network
    G::LightGraphs.SimpleGraphs.SimpleGraph{Int}
    node_status::Vector{Int}
    deltaA::Vector{Float64}
    deltaB::Vector{Float64}
    qA::Vector{Float64}
    qB::Vector{Float64}
end

In [5]:
function reset!(N::Network)
    N.node_status = zeros(Int, nv(N.G))
    return nothing
end

reset! (generic function with 1 method)

In [6]:
function active_nbrs(N::Network, node::Int, brand::Int)
    n_active_nbrs = 0
    
    for nbr in neighbors(N.G, node)
        if N.node_status[nbr] == brand
            n_active_nbrs += 1
        end
    end
    
    return n_active_nbrs
end

active_nbrs (generic function with 1 method)

In [7]:
function adopt_prob(N::Network, node::Int)
    n_active_nbrsA = active_nbrs(N, node, 1)
    n_active_nbrsB = active_nbrs(N, node, 2)
    
    pA_i = 1 - (1 - N.deltaA[node]) * ((1 - N.qA[node])^n_active_nbrsA)
    pB_i = 1 - (1 - N.deltaB[node]) * ((1 - N.qB[node])^n_active_nbrsB)
    alpha = pA_i/(pA_i+pB_i)
    
    pA = pA_i * (1 - pB_i) + alpha * pA_i * pB_i
    pB = pB_i * (1 - pA_i) + (1 - alpha) * pA_i * pB_i 

    return pA, pB
end

adopt_prob (generic function with 1 method)

In [13]:
function seed!(N::Network, seed_size::Int, brand::Int, mode::String)
    if mode == "random" # random seeding
        seed = sample(vertices(N.G), seed_size, replace=false)
        N.node_status[seed] = brand
    elseif mode == "hub" # centrality seeding
        seed = sortperm(degree_centrality(N.G), rev=true)[1:seed_size]
        N.node_status[seed] = brand
    else # expert seeding
        if brand == 1
            seed = sortperm(N.qA, rev=true)[1:seed_size]
            N.node_status[seed] = brand
        else
            seed = sortperm(N.qB, rev=true)[1:seed_size]
            N.node_status[seed] = brand
        end
    end
    
    return nothing
end     

seed! (generic function with 2 methods)

In [10]:
function evolve!(N::Network)
    for node in shuffle(vertices(N.G))
        if N.node_status[node] == 0 # If not adopted
            r = rand(Uniform())
            pA, pB = adopt_prob(N, node)
            
            if r < pA
                N.node_status[node] = 1
            end
            
            if r < pB
                N.node_status[node] = 2
            end
        end
    end
    
    return nothing
end

evolve! (generic function with 1 method)

In [17]:
function simulate(n::Int, k::Int, p::Float64; T = 30, n_realizations = 20, sigma = 0.02)
    parameter_space = [(deltaA, deltaB, qA, qB, seed_size) for deltaA in linspace(0.001, 0.02, 5),
                                                               deltaB in linspace(0.001, 0.02, 5),
                                                               qA in linspace(0.04, 0.16, 5),
                                                               qB in linspace(0.04, 0.16, 5),
                                                               seed_size in linspace(0.005, 0.05, 5)]
    
    output = DataFrame(r = Int[], t = Int[], seed_size = Float64[], 
                       deltaA = Float64[], deltaB = Float64[], 
                       qA = Float64[], qB = Float64[],
                       engaged_randA = Int[], engaged_randB = Int[], 
                       engaged_hubA = Int[], engaged_hubB = Int[],
                       engaged_expertA = Int[], engaged_expertB = Int[])
    
    
    @showprogress 1 "Simulating..." for (deltaA, deltaB, qA, qB, seed_size) in parameter_space
        for r in 1:n_realizations
            g = watts_strogatz(n, k, p)
            N = Network(g, zeros(Int, n), fill!(Vector{Float64}(n), deltaA), fill!(Vector{Float64}(n), deltaB),
                        rand(TruncatedNormal(qA, sigma, 0, Inf), n), rand(TruncatedNormal(qB, sigma, 0, Inf), n))
            
            # randomly seeding for brand A and evolution
            seed!(N, floor(Int, seed_size * n), 1, "random")
            for t in 1:T
                evolve!(N)
                n_engaged = sum(N.node_status .!= 0)
                push!(output, [r, t, seed_size, deltaA, deltaB, qA, qB, n_engaged, 0, 0, 0, 0, 0])
            end
            
            reset!(N)
            
            # randomly seeding for brand B and evolution
            seed!(N, floor(Int, seed_size * n), 2, "random")
            for t in 1:T
                evolve!(N)
                n_engaged = sum(N.node_status .!= 0)
                push!(output, [r, t, seed_size, deltaA, deltaB, qA, qB, 0, n_engaged, 0, 0, 0, 0])
            end
            
            reset!(N)
            
            # seeding hubs for brand A and evolution
            seed!(N, floor(Int, seed_size * n), 1, "hub")
            for t in 1:T
                evolve!(N)
                n_engaged = sum(N.node_status .!= 0)
                push!(output, [r, t, seed_size, deltaA, deltaB, qA, qB, 0, 0, n_engaged, 0, 0, 0])
            end
            
            reset!(N)
            
            # seeding hubs for brand B and evolution
            seed!(N, floor(Int, seed_size * n), 2, "hub")
            for t in 1:T
                evolve!(N)
                n_engaged = sum(N.node_status .!= 0)
                push!(output, [r, t, seed_size, deltaA, deltaB, qA, qB, 0, 0, 0, n_engaged, 0, 0])
            end
            
            reset!(N)
            
            # seeding experts for brand A and evolution
            seed!(N, floor(Int, seed_size * n), 1, "expert")
            for t in 1:T
                evolve!(N)
                n_engaged = sum(N.node_status .!= 0)
                push!(output, [r, t, seed_size, deltaA, deltaB, qA, qB, 0, 0, 0, 0, n_engaged, 0])
            end
            
            reset!(N)
            
            # seeding experts for brand B and evolution
            seed!(N, floor(Int, seed_size * n), 2, "expert")
            for t in 1:T
                evolve!(N)
                n_engaged = sum(N.node_status .!= 0)
                push!(output, [r, t, seed_size, deltaA, deltaB, qA, qB, 0, 0, 0, 0, 0, n_engaged])
            end
            
        end
        
    end
    
    return output
    
end    

simulate (generic function with 1 method)

In [18]:
result = simulate(1000, 4, 0.5)

Simulating...100%|██████████████████████████████████████| Time: 0:12:04


,r,t,seed_size,deltaA,deltaB,qA,qB,engaged_randA,engaged_randB,engaged_hubA,engaged_hubB,engaged_expertA,engaged_expertB
1,1,1,0.005,0.001,0.001,0.04,0.04,5,0,0,0,0,0
2,1,2,0.005,0.001,0.001,0.04,0.04,6,0,0,0,0,0
3,1,3,0.005,0.001,0.001,0.04,0.04,6,0,0,0,0,0
4,1,4,0.005,0.001,0.001,0.04,0.04,7,0,0,0,0,0
5,1,5,0.005,0.001,0.001,0.04,0.04,8,0,0,0,0,0
6,1,6,0.005,0.001,0.001,0.04,0.04,9,0,0,0,0,0
7,1,7,0.005,0.001,0.001,0.04,0.04,13,0,0,0,0,0
8,1,8,0.005,0.001,0.001,0.04,0.04,14,0,0,0,0,0
9,1,9,0.005,0.001,0.001,0.04,0.04,17,0,0,0,0,0
10,1,10,0.005,0.001,0.001,0.04,0.04,19,0,0,0,0,0


In [21]:
size(result)

(11250000, 13)

In [22]:
writetable("value-WOM.csv", result)